# MyGPT

MyGPT is a next-character predictor. Given a sequence of characters, it is able to predict the next character

```python

num_chars_to_predict = 7

prompt = ['s', 'u', 'b', 't']

for _ in range(num_chars_to_predict):
    prediction = mygpt(prompt)
    prompt.append(prediction)
    print(prompt)

# ['s', 'u', 'b', 't', 'r']
# ['s', 'u', 'b', 't', 'r', 'a']
# ['s', 'u', 'b', 't', 'r', 'a', 'c']
# ['s', 'u', 'b', 't', 'r', 'a', 'c', 't']
# ['s', 'u', 'b', 't', 'r', 'a', 'c', 't', 'i']
# ['s', 'u', 'b', 't', 'r', 'a', 'c', 't', 'i', 'o']
# ['s', 'u', 'b', 't', 'r', 'a', 'c', 't', 'i', 'o', 'n']
```

In [1]:
import torch
import torch.nn.functional as F
import os

In [2]:
# determine which device to perform training on: CPU or GPU

device = "cuda" if torch.cuda.is_available() else "cpu"

Here we load a dataset of calculus lectures. MyGPT will be trained to generate text that resembles these lectures

In [3]:
# load the training data as raw text

from MyGPT.pretrain import get_data, get_train_val_data
from MyGPT.vocab import Tokenizer, create_vocabulary

data_filename = "calculus.txt"
data_path = os.path.join("data", data_filename)
raw_data = get_data(data_path)

# create a vocabulary of all the unique characters in the raw text

vocab, vocab_size = create_vocabulary(raw_data)
tokenizer = Tokenizer(vocab)

# tokenize the training data to be tensors of individual characters

train_data, val_data = get_train_val_data(raw_data, tokenizer, device)

Here we initialize the MyGPT model. It is able to keep 64 characters in its "working memory" at a time. This is called its `context_length`. The model uses this context to predict the most likely characters to come next

In [4]:
# initialize the MyGPT model

from MyGPT.transformer import Transformer as MyGPT

context_length = 64  # the max number of characters that MyGPT can keep in its "working memory"

mygpt = MyGPT(
    vocab_size,
    device,
    context_length=context_length,
    d_embed=128,
    n_head=8,
    n_layer=4,
)
mygpt.to(device);


In [5]:
# initialize the training hyperparameters

batch_size = 16
max_iters = 5000
eval_interval = 500
eval_iters = 100
learning_rate = 1e-3

# initialize the optimizer

optimizer = torch.optim.AdamW(mygpt.parameters(), lr=learning_rate)

Here we begin the training loop for MyGPT. Notice how at the beginning of the loop, MyGPT produces unreadable text. But as the training continues, words start to form and the text is more human-like

The training loop works by performing the following computations:
1. Take in an input context of 64 characters and produce predictions
2. Measure how incorrect the predictions are, which we call the `loss`
3. Compute the gradient of the `loss` with respect to the model parameters
5. Update the model parameters in the direction of negative `loss`, to minimize the `loss`

As the `loss` gets minimized, MyGPT's predictions become more correct

In [6]:
from MyGPT.pretrain import estimate_loss, get_batch
from MyGPT.generate import generate

for iteration in range(max_iters):
    if iteration % eval_interval == 0 or iteration == max_iters - 1:
        train_loss = estimate_loss(
            mygpt, train_data, batch_size, context_length, eval_iters
        )

        print("\n================================================================")
        print(
            "iteration: {} | loss: {:0.3f}".format(
                iteration, train_loss
            )
        )
        print("================================================================\n")

        context = torch.tensor([[0]], dtype=torch.long, device=device)
        generate(mygpt, context, tokenizer, num_new_tokens=200)

    x, y = get_batch(train_data, batch_size, context_length)
    _, loss = mygpt(x, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



iteration: 0 | loss: 4.745

Ak7/m’)o"=TAXL?]5|$mg0m.’ZM_?3zw|@pif]|1\v1"J,s;kQ$l2 %"�j4HyQlJEl^h'nA(#c.U.Z[X!n5@2Hh7zZ@(C O=(6-K°5wjSh7G
p8.CXWX2.uQvWHdZwMb°/0/M`.YQEF_X�|g7B7v^’-he\.M+`lV_/??L+M�[*qhb/M'5["O6X9Y(zWl$s–|'4p-/$KO

iteration: 500 | loss: 1.953

of there iffrod expulich tant opersenntord. So same
clobets culd. We
dext so we tee mantidinstientied to sef destrse pund faind mont. This the. OK,
ican| The is onation,
those an to the od hon42, 1)OR

iteration: 1000 | loss: 1.647

firct, minu puts this one rows chectormuld ain in Bis
theire's which in mattrijed becaused you an
looke of you, in but A/3 I not causes, H. So larke, ormpluen you
core of thinkat wesorvally that's smo

iteration: 1500 | loss: 1.521

roops that node deleta it, just that sarch the if you
not that the matrix, and graphances darcyn hand bacturnally negarigral
och that the flater the catrition, area sumbicular. AUDIBLking used
the b, 

iteration: 2000 | loss: 1.459

In see, this lareg you relar. I want to 

Now that the model has been trained, let's input a text prompt into MyGPT and have it generate some text

The prompt is set to "multiplic". Let's see how it completes the word

We then let it continue generating a total of 2000 characters

In [7]:
from MyGPT.generate import generate

prompt = "multiplic"

# encode the prompt into a tensor that MyGPT is able to process

prompt = tokenizer.encode(prompt)
prompt = torch.tensor(prompt, device=device).unsqueeze(0)

generate(mygpt, prompt, tokenizer, num_new_tokens=2000)

ation. And work anothe one cut by that
this one is in A this is a
quite using table inside case,
if this this is a two address are at set call thems. So many now monically, one way D
or over error-- you're going to a here, and I'll call remememement, say, we've got eScall YTo kneut
that this length this list t. That's time very the Containue,
that's from the operations and NP m be tree. We is a nece"ece a number
of that we're going to be numerame being that. A first, link where this -- I'm call happen to
and then this by testal over go here. Jut again. If you still justhey obsolute, or convergeticula
ere to one you aften good. FRion If it was that
people troubles where I in one jeck
in the orlar of jushe other quinds that these And so smillion
is a be quiz, and num=ratory, compares. And it kin
equal to how week you make to a unique 90, this thing. Because they doubres. Fallon you changing A. So minus b 0; threshat the
elements we will see we get underscoptive
a cool formula to work of 